### Introduction

### Data Understanding

We will make use of datasets from the Chicago Data Portal, an open data resource where one may find relevant information about the city. In particular we will be focusing on three datasets:Traffic Crashes - Vehicles, Traffic Crashes - People and Traffic Crashes - Crashes. 
 
**Dataset 1: CRASH Data**

Description of Columns:  
Some of the columns found in the dataset are:

>'CRASH_RECORD_ID':
This number can be used to link to the same crash in the Vehicles and People datasets. This number also serves as a unique ID in this dataset.  

>CRASH_DATE:
Date and time of crash as entered by the reporting officer

>WEATHER_CONDITION:
Weather condition at time of crash, as determined by reporting officer

>ROADWAY_SURFACE_COND:
Road surface condition, as determined by reporting officer

>CRASH_TYPE:
A general severity classification for the crash. Can be either Injury and/or Tow Due to Crash or No Injury / Drive Away

The full description of the columns can be found here or the columns_desc.txt file.

Insights:

**Dataset 2: Vehicles Data**

Description of Columns:  
Some of the columns found in the dataset are:

>'CRASH_RECORD_ID':
This number can be used to link to the same crash in the Vehicles and People datasets. This number also serves as a unique ID in this dataset.  

>CRASH_UNIT_ID	
A unique identifier for each vehicle record.

>MAKE	
The make (brand) of the vehicle, if relevant

>VEHICLE_USE	
The normal use of the vehicle, if relevant

>EXCEED_SPEED_LIMIT_I	
Indicator of whether the unit was speeding, as determined by the reporting officer

The full description of the columns can be found here or the columns_desc.txt file.

Insights: 
Passanger vehicles used for personal use are more prone to accidents.We can also see that toyota and chevrolet makes are leading in number of accidents with 118 and 114 respectfully.

**Dataset 3: People Data**

Description of Columns:  
Some of the columns found in the dataset are:

>'CRASH_RECORD_ID':
This number can be used to link to the same crash in the Vehicles and People datasets. This number also serves as a unique ID in this dataset.  

>PERSON_ID	
A unique identifier for each person record. IDs starting with P indicate passengers. IDs starting with O indicate a person who was not a passenger in the vehicle (e.g., driver, pedestrian, cyclist, etc.).

>PERSON_TYPE	
Type of roadway user involved in crash

>SEX	
Gender of person involved in crash, as determined by reporting officer

>AGE	
Age of person involved in crash

The full description of the columns can be found here or the columns_desc.txt file.

Insights:
Drivers are more prone to be affected with the accidents with 731 cases.
Most victims are Males.
The most affected age group is between 26-30.



### Data Cleaning

In [25]:
#Importing the relevant Libraries
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

In [26]:
# Deriving the data from
pd.set_option('display.max_columns', None)
crash = pd.read_csv('https://data.cityofchicago.org/resource/85ca-t3if.csv')
vehicle = pd.read_csv('https://data.cityofchicago.org/resource/68nd-jvt3.csv')
person = pd.read_csv('https://data.cityofchicago.org/resource/u6pd-qa9d.csv')

In [27]:
# merging all three databases into 1, observing shape and previewing data 
merged = pd.merge(left=crash, right = vehicle, left_on='crash_record_id', right_on="crash_record_id")
df = pd.merge(left=merged, right=person, left_on = 'vehicle_id', right_on='vehicle_id')
print(df.shape)
df.head()

(1575, 149)


,crash_record_id_x,rd_no_x,crash_date_est_i,crash_date_x,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,lane_cnt,alignment,roadway_surface_cond,road_defect,report_type,crash_type,intersection_related_i,private_property_i,hit_and_run_i,damage,date_police_notified,prim_contributory_cause,sec_contributory_cause,street_no,street_direction,street_name,beat_of_occurrence,photos_taken_i,statements_taken_i,dooring_i,work_zone_i,work_zone_type,workers_present_i,num_units,most_severe_injury,injuries_total,injuries_fatal,injuries_incapacitating,injuries_non_incapacitating,injuries_reported_not_evident,injuries_no_indication,injuries_unknown,crash_hour,crash_day_of_week,crash_month,latitude,longitude,location,crash_unit_id,rd_no_y,crash_date_y,unit_no,unit_type,num_passengers,vehicle_id,cmrc_veh_i,make,model,lic_plate_state,vehicle_year,vehicle_defect,vehicle_type,vehicle_use,travel_direction,maneuver,towed_i,fire_i,occupant_cnt,exceed_speed_limit_i,towed_by,towed_to,area_00_i,area_01_i,area_02_i,area_03_i,area_04_i,area_05_i,area_06_i,area_07_i,area_08_i,area_09_i,area_10_i,area_11_i,area_12_i,area_99_i,first_contact_point,cmv_id,usdot_no,ccmc_no,ilcc_no,commercial_src,gvwr,carrier_name,carrier_state,carrier_city,hazmat_placards_i,hazmat_name,un_no,hazmat_present_i,hazmat_report_i,hazmat_report_no,mcs_report_i,mcs_report_no,hazmat_vio_cause_crash_i,mcs_vio_cause_crash_i,idot_permit_no,wide_load_i,trailer1_width,trailer2_width,trailer1_length,trailer2_length,total_vehicle_length,axle_cnt,vehicle_config,cargo_body_type,load_type,hazmat_out_of_service_i,mcs_out_of_service_i,hazmat_class,person_id,person_type,crash_record_id_y,rd_no,crash_date,seat_no,city,state,zipcode,sex,age,drivers_license_state,drivers_license_class,safety_equipment,airbag_deployed,ejection,injury_classification,hospital,ems_agency,ems_run_no,driver_action,driver_vision,physical_condition,pedpedal_action,pedpedal_visibility,pedpedal_location,bac_result,bac_result_value,cell_phone_use
0,b3235541759a29563e1f11d164de82b3b82ab366e5477d...,NaN,NaN,2023-08-06T22:05:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,NaN,NaN,NaN,"OVER $1,500",2023-08-06T22:30:00.000,FOLLOWING TOO CLOSELY,NOT APPLICABLE,4300,W,IRVING PARK RD,1722,NaN,NaN,NaN,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0,0,0,0,0,2,0,22,1,8,41.953668,-87.735508,POINT (-87.735507825353 41.953667894495),1633936,NaN,2023-08-06T22:05:00.000,1,DRIVER,NaN,1554848.0,NaN,JEEP,COMPASS,IL,2018.0,UNKNOWN,SPORT UTILITY VEHICLE (SUV),PERSONAL,W,STRAIGHT AHEAD,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,FRONT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O1633936,DRIVER,b3235541759a29563e1f11d164de82b3b82ab366e5477d...,NaN,2023-08-06T22:05:00.000,NaN,CHICAGO,IL,60641.0,M,0.0,IL,D,USAGE UNKNOWN,NOT APPLICABLE,NONE,NO INDICATION OF INJURY,NaN,NaN,NaN,FOLLOWED TOO CLOSELY,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,b3235541759a29563e1f11d164de82b3b82ab366e5477d...,NaN,NaN,2023-08-06T22:05:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,NaN,NaN,NaN,"OVER $1,500",2023-08-06T22:30:00.000,FOLLOWING TOO CLOSELY,NOT APPLICABLE,4300,W,IRVING PARK RD,1722,NaN,NaN,NaN,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0,0,0,0,0,2,0,22,1,8,41.953668,-87.735508,POINT (-87.735507825353 41.953667894495),1633937,NaN,2023-08-06T22:05:00.000,2,DRIVER,NaN,1554864.0,NaN,HONDA,ODYSSEY,IL,2005.0,NONE,SPORT UTILITY VEHICLE (SUV),PERSONAL,W,SLOW/STOP IN TRAFFIC,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [28]:
# calculating nulls percentage
nulls = df.isna().sum()
null_percent = nulls[nulls>0] / len(df)

In [29]:
# Extracting columns with excesssive nulls which is set at 80%
Index_label = null_percent[null_percent>.80].index.tolist()
Index_label[:5]

['rd_no_x',
 'crash_date_est_i',
 'lane_cnt',
 'private_property_i',
 'photos_taken_i']

In [30]:
# previewing shape, data and info 
df = df.drop(columns = Index_label)
print(df.shape)
display(df.head())
df.info()

(1575, 82)


,crash_record_id_x,crash_date_x,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,road_defect,report_type,crash_type,intersection_related_i,hit_and_run_i,damage,date_police_notified,prim_contributory_cause,sec_contributory_cause,street_no,street_direction,street_name,beat_of_occurrence,num_units,most_severe_injury,injuries_total,injuries_fatal,injuries_incapacitating,injuries_non_incapacitating,injuries_reported_not_evident,injuries_no_indication,injuries_unknown,crash_hour,crash_day_of_week,crash_month,latitude,longitude,location,crash_unit_id,crash_date_y,unit_no,unit_type,num_passengers,vehicle_id,make,model,lic_plate_state,vehicle_year,vehicle_defect,vehicle_type,vehicle_use,travel_direction,maneuver,occupant_cnt,area_01_i,area_12_i,first_contact_point,person_id,person_type,crash_record_id_y,crash_date,city,state,zipcode,sex,age,drivers_license_state,drivers_license_class,safety_equipment,airbag_deployed,ejection,injury_classification,hospital,ems_agency,driver_action,driver_vision,physical_condition,pedpedal_action,pedpedal_visibility,pedpedal_location,bac_result
0,b3235541759a29563e1f11d164de82b3b82ab366e5477d...,2023-08-06T22:05:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,NaN,NaN,"OVER $1,500",2023-08-06T22:30:00.000,FOLLOWING TOO CLOSELY,NOT APPLICABLE,4300,W,IRVING PARK RD,1722,2,NO INDICATION OF INJURY,0,0,0,0,0,2,0,22,1,8,41.953668,-87.735508,POINT (-87.735507825353 41.953667894495),1633936,2023-08-06T22:05:00.000,1,DRIVER,NaN,1554848.0,JEEP,COMPASS,IL,2018.0,UNKNOWN,SPORT UTILITY VEHICLE (SUV),PERSONAL,W,STRAIGHT AHEAD,1.0,NaN,Y,FRONT,O1633936,DRIVER,b3235541759a29563e1f11d164de82b3b82ab366e5477d...,2023-08-06T22:05:00.000,CHICAGO,IL,60641.0,M,0.0,IL,D,USAGE UNKNOWN,NOT APPLICABLE,NONE,NO INDICATION OF INJURY,NaN,NaN,FOLLOWED TOO CLOSELY,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED
1,b3235541759a29563e1f11d164de82b3b82ab366e5477d...,2023-08-06T22:05:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,NaN,NaN,"OVER $1,500",2023-08-06T22:30:00.000,FOLLOWING TOO CLOSELY,NOT APPLICABLE,4300,W,IRVING PARK RD,1722,2,NO INDICATION OF INJURY,0,0,0,0,0,2,0,22,1,8,41.953668,-87.735508,POINT (-87.735507825353 41.953667894495),1633937,2023-08-06T22:05:00.000,2,DRIVER,NaN,1554864.0,HONDA,ODYSSEY,IL,2005.0,NONE,SPORT UTILITY VEHICLE (SUV),PERSONAL,W,SLOW/STOP IN TRAFFIC,1.0,NaN,NaN,REAR,O1633937,DRIVER,b3235541759a29563e1f11d164de82b3b82ab366e5477d...,2023-08-06T22:05:00.000,CHICAGO,IL,60634.0,M,57.0,IL,D,USAGE UNKNOWN,NOT APPLICABLE,NONE,NO INDICATION OF INJURY,NaN,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED
2,49bc94e87a5392c517b613490383374bec0d76c7d4e283...,2023-08-06T21:46:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,NO INJURY / DRIVE AWAY,Y,NaN,"OVER $1,500",2023-08-06T21:46:00.000,UNABLE TO DETERMINE,UNABLE TO DETERMINE,7900,S,RACINE AVE,612,2,NO INDICATION OF INJURY,0,0,0,0,0,3,0,21,1,8,41.750475,-87.653883,POINT (-87.653883334391 41.750474929778),1633919,2023-08-06T21:46:00.000,1,DRIVER,1.0,1554825.0,KIA,OPTIMA / K5,IL,2022.0,NONE,PASSENGER,PERSONAL,N,STRAIGHT AHEAD,2.0,Y,Y,FRONT,O1633919,DRIVER,49bc94e87a5392c517b613490383374bec0d76c7d4e283...,2023-08-06T21:46:00.000,CHICAGO,IL,60620.0,F,53.0,IL,D,USAGE UNKNOWN,DID NOT DEPLOY,NONE,NO INDICATION OF INJURY,NaN,NaN,UNKNOWN,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED
3,49bc94e87a5392c517b613490383374bec0d76c7d4e283...,2023-08-06T21:46:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,NO INJURY / DRIVE AWAY,Y,NaN,

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1575 entries, 0 to 1574
Data columns (total 82 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   crash_record_id_x              1575 non-null   object 
 1   crash_date_x                   1575 non-null   object 
 2   posted_speed_limit             1575 non-null   int64  
 3   traffic_control_device         1575 non-null   object 
 4   device_condition               1575 non-null   object 
 5   weather_condition              1575 non-null   object 
 6   lighting_condition             1575 non-null   object 
 7   first_crash_type               1575 non-null   object 
 8   trafficway_type                1575 non-null   object 
 9   alignment                      1575 non-null   object 
 10  roadway_surface_cond           1575 non-null   object 
 11  road_defect                    1575 non-null   object 
 12  report_type                    1570 non-null   o

In [32]:
# dropping following columns due to irrelevance in predicting the cause of car accidents 
# irrelevent columns were dropped due to column description
drop = ['report_type', 'crash_type', 'damage', 'date_police_notified', 'injuries_fatal', 
        'injuries_incapacitating', 'most_severe_injury','injuries_non_incapacitating',
        'injuries_reported_not_evident', 'injuries_no_indication', 'injuries_unknown', 'crash_date',
        'crash_date', 'ejection','injury_classification','intersection_related_i','beat_of_occurrence',
        'bac_result','pedpedal_location']

df = df.drop(columns = drop)
print(df.shape)
df.head()

(1575, 64)


,crash_record_id_x,crash_date_x,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,road_defect,hit_and_run_i,prim_contributory_cause,sec_contributory_cause,street_no,street_direction,street_name,num_units,injuries_total,crash_hour,crash_day_of_week,crash_month,latitude,longitude,location,crash_unit_id,crash_date_y,unit_no,unit_type,num_passengers,vehicle_id,make,model,lic_plate_state,vehicle_year,vehicle_defect,vehicle_type,vehicle_use,travel_direction,maneuver,occupant_cnt,area_01_i,area_12_i,first_contact_point,person_id,person_type,crash_record_id_y,city,state,zipcode,sex,age,drivers_license_state,drivers_license_class,safety_equipment,airbag_deployed,hospital,ems_agency,driver_action,driver_vision,physical_condition,pedpedal_action,pedpedal_visibility
0,b3235541759a29563e1f11d164de82b3b82ab366e5477d...,2023-08-06T22:05:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,FOLLOWING TOO CLOSELY,NOT APPLICABLE,4300,W,IRVING PARK RD,2,0,22,1,8,41.953668,-87.735508,POINT (-87.735507825353 41.953667894495),1633936,2023-08-06T22:05:00.000,1,DRIVER,NaN,1554848.0,JEEP,COMPASS,IL,2018.0,UNKNOWN,SPORT UTILITY VEHICLE (SUV),PERSONAL,W,STRAIGHT AHEAD,1.0,NaN,Y,FRONT,O1633936,DRIVER,b3235541759a29563e1f11d164de82b3b82ab366e5477d...,CHICAGO,IL,60641.0,M,0.0,IL,D,USAGE UNKNOWN,NOT APPLICABLE,NaN,NaN,FOLLOWED TOO CLOSELY,UNKNOWN,UNKNOWN,NaN,NaN
1,b3235541759a29563e1f11d164de82b3b82ab366e5477d...,2023-08-06T22:05:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,FOLLOWING TOO CLOSELY,NOT APPLICABLE,4300,W,IRVING PARK RD,2,0,22,1,8,41.953668,-87.735508,POINT (-87.735507825353 41.953667894495),1633937,2023-08-06T22:05:00.000,2,DRIVER,NaN,1554864.0,HONDA,ODYSSEY,IL,2005.0,NONE,SPORT UTILITY VEHICLE (SUV),PERSONAL,W,SLOW/STOP IN TRAFFIC,1.0,NaN,NaN,REAR,O1633937,DRIVER,b3235541759a29563e1f11d164de82b3b82ab366e5477d...,CHICAGO,IL,60634.0,M,57.0,IL,D,USAGE UNKNOWN,NOT APPLICABLE,NaN,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN
2,49bc94e87a5392c517b613490383374bec0d76c7d4e283...,2023-08-06T21:46:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,UNABLE TO DETERMINE,UNABLE TO DETERMINE,7900,S,RACINE AVE,2,0,21,1,8,41.750475,-87.653883,POINT (-87.653883334391 41.750474929778),1633919,2023-08-06T21:46:00.000,1,DRIVER,1.0,1554825.0,KIA,OPTIMA / K5,IL,2022.0,NONE,PASSENGER,PERSONAL,N,STRAIGHT AHEAD,2.0,Y,Y,FRONT,O1633919,DRIVER,49bc94e87a5392c517b613490383374bec0d76c7d4e283...,CHICAGO,IL,60620.0,F,53.0,IL,D,USAGE UNKNOWN,DID NOT DEPLOY,NaN,NaN,UNKNOWN,NOT OBSCURED,NORMAL,NaN,NaN
3,49bc94e87a5392c517b613490383374bec0d76c7d4e283...,2023-08-06T21:46:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,UNABLE TO DETERMINE,UNABLE TO DETERMINE,7900,S,RACINE AVE,2,0,21,1,8,41.750475,-87.653883,POINT (-87.653883334391 41.750474929778),1633919,2023-08-06T21:46:00.000,1,DRIVER,1.0,1554825.0,KIA,OPTIMA / K5,IL,2022.0,NONE,PASSENGER,PERSONAL,N,STRAIGHT AHEAD,2.0,Y,Y,FRONT,P361625,PASSENGER,49bc94e87a5392c517b613490383374bec0d76c7d4e283...,CHIACGO,IL,60620.0,F,48.0,NaN,NaN,USAGE UNKNOWN,NOT APPLICABLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,49bc94e87a5392c517b613490383374bec0d76c7d4e283...,2023-08-06T21:46:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,UNABLE TO DETERMINE,UNABLE TO DETERMINE,7900,S,RACINE AVE,2,0,21,1,8,41.750475,-87.653883,POINT (-87.653883334391 41.750474929778),1633920,2023-08-06T21:46:00.000,2,DRIVER,NaN,1554826.0,BUICK,VERANO,IL,2013.0,NONE,PASSENGER,PERSONAL,E,STRAIGHT AHEAD,1.0,NaN,NaN,SIDE-RIGHT-REAR,O1633920,DRIVER,49bc94e87a5392c517b613490383374bec0

In [34]:
df.columns

Index(['crash_record_id_x', 'crash_date_x', 'posted_speed_limit',
       'traffic_control_device', 'device_condition', 'weather_condition',
       'lighting_condition', 'first_crash_type', 'trafficway_type',
       'alignment', 'roadway_surface_cond', 'road_defect', 'hit_and_run_i',
       'prim_contributory_cause', 'sec_contributory_cause', 'street_no',
       'street_direction', 'street_name', 'num_units', 'injuries_total',
       'crash_hour', 'crash_day_of_week', 'crash_month', 'latitude',
       'longitude', 'location', 'crash_unit_id', 'crash_date_y', 'unit_no',
       'unit_type', 'num_passengers', 'vehicle_id', 'make', 'model',
       'lic_plate_state', 'vehicle_year', 'vehicle_defect', 'vehicle_type',
       'vehicle_use', 'travel_direction', 'maneuver', 'occupant_cnt',
       'area_01_i', 'area_12_i', 'first_contact_point', 'person_id',
       'person_type', 'crash_record_id_y', 'city', 'state', 'zipcode', 'sex',
       'age', 'drivers_license_state', 'drivers_license_class',